## Seed riders for new event

This Notebook is to utilize the ranked riders data and load a list of participating riders in a new event.
The riders in the new event will get seeded if they have been participating in the competition from last year's Vintercupen.

In [20]:
import os
import pandas as pd
import datetime

## Load previous ranked data

In [21]:
file_path = '../src/ranked/ranked.csv'

# Read the CSV file into a DataFrame
ranked_dataframe = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(ranked_dataframe.head())

                       Namn            Klubb     #         Klass  \
0  Simon Hopander Ankarhake     Nyköpings MS   304  Motion 16-29   
1            Sören Fransson         Kinds MK  1236  Motion 50-59   
2            Niklas Persson     Karlskoga EK    28        Senior   
3            Rikard Hansson         Tibro MK    73        Senior   
4            Jimmy Ivarsson  SMK Trollhättan   559  Motion 40-49   

   Rank FMCK Skövde  Rank Töreboda MK  Rank Tidaholms MK  Rank Carlsborgs MK  \
0               NaN               NaN                NaN                 NaN   
1               NaN               NaN                1.0               161.0   
2               NaN               2.0                NaN                 NaN   
3               NaN               3.0                NaN                 2.0   
4               NaN               NaN                NaN                 NaN   

   Rank Tibro MK  Rank Falköpings MK  AvgRank  
0            NaN                 1.0      1.0  
1            N

## Load participants for an upcoming event.

In [22]:
file_path = '../src/unseeded/unseeded.csv'

# Read the CSV file into a DataFrame with the second row as column names
unseeded_dataframe = pd.read_csv(file_path, encoding='cp1252', header=0)

# Display the first few rows of the DataFrame
print(unseeded_dataframe.head(-10))

     Tidaholms MK  2024    #                Namn            Klubb      Märke  \
0             NaN   NaN  200    Andreas Toresson  Lilla Edets MCK        KTM   
1             NaN   NaN  213         Björn Levin         Kinna MK  Husqvarna   
2             NaN   NaN  241     Joakim Svensson    Linköpings MS        KTM   
3             NaN   NaN  106      Wilmer Jansson     Tidaholms MK  Husqvarna   
4             NaN   NaN  119      Hugo Andersson         Tibro MK     Sherco   
..            ...   ...  ...                 ...              ...        ...   
168           NaN   NaN  102          Liam Qvist       Åbågen MCK      Honda   
169           NaN   NaN  103     Anton Magnusson  Norrahammars MK        KTM   
170           NaN   NaN  104        Lisa Nyqvist  Stenungsunds MS        KTM   
171           NaN   NaN  106    Hampus Andersson         Tibro MK     Sherco   
172           NaN   NaN  113  Benjamin Bengtsson    Falköpings MK  Husqvarna   

         Klass  
0        Bredd  
1    

## Merge data.
Merging data from the two files to set a seeding to the riders in the unseeded dataframe.

In [23]:
# Merge or lookup operation to find the AvgRank from ranked_dataframe for each row in unseeded_dataframe
seeded_df = unseeded_dataframe.merge(
    ranked_dataframe[['Namn', 'Klubb', 'AvgRank']],  # Selecting necessary columns from ranked_dataframe
    on=['Namn', 'Klubb'],  # Merging on 'Namn' and 'Klubb' columns
    how='left'  # Perform a left join to keep all rows from unseeded_dataframe
)

# Drop the 'Unnamed: 5' column if it exists
if 'Unnamed: 5' in seeded_df.columns:
    seeded_df.drop(columns=['Unnamed: 5'], inplace=True)

# Rename the 'AvgRank' column to 'Seeding'
seeded_df.rename(columns={'AvgRank': 'Seeding'}, inplace=True)

# Find the maximum AvgRank from ranked_dataframe
max_avg_rank = ranked_dataframe['AvgRank'].max()

# Fill missing Seeding values with the maximum AvgRank plus 1
seeded_df['Seeding'].fillna(max_avg_rank + 1, inplace=True)

# Sort by Seeding column
seeded_df = seeded_df.sort_values(by='Seeding')

seeded_df.head(-10)

,Tidaholms MK,2024,#,Namn,Klubb,Märke,Klass,Seeding
87,NaN,NaN,1218,Sören Fransson,Kinds MK,KTM,Motion 50-59,1.00
148,NaN,NaN,20,Robert Friberg,FMCK Skövde,Husqvarna,Senior,7.60
147,NaN,NaN,3,Nisse Bengtsson,Tidaholms MK,Yamaha,Senior,7.75
4,NaN,NaN,119,Hugo Andersson,Tibro MK,Sherco,Junior,15.00
149,NaN,NaN,82,Arvid Karlsson,FMCK Skövde,KTM,Senior,15.20
...,...,...,...,...,...,...,...,...
99,NaN,NaN,1235,Patrik Högberg,Lilla Edets MCK,KTM,Motion 50-59,422.00
10,NaN,NaN,188,Alexander Wallberg,Älvbygdens MK,Husqvarna,Junior,422.00
181,NaN,NaN,186,Felix Johansson,Skene MS,Yamaha,Ungdom E1,422.00
16,NaN,NaN,505,Viktor Blom,Skene MS,Yamaha,Motion -39,422.00


## Set seeding number.
To sort the seeding in a ascending number startin from 1, set the row number for each row in the Seeding column.

In [24]:
seeded_df['Seeding'] = seeded_df.reset_index(drop=True).index + 1

seeded_df.head(100)

,Tidaholms MK,2024,#,Namn,Klubb,Märke,Klass,Seeding
87,NaN,NaN,1218,Sören Fransson,Kinds MK,KTM,Motion 50-59,1
148,NaN,NaN,20,Robert Friberg,FMCK Skövde,Husqvarna,Senior,2
147,NaN,NaN,3,Nisse Bengtsson,Tidaholms MK,Yamaha,Senior,3
4,NaN,NaN,119,Hugo Andersson,Tibro MK,Sherco,Junior,4
149,NaN,NaN,82,Arvid Karlsson,FMCK Skövde,KTM,Senior,5
...,...,...,...,...,...,...,...,...
141,NaN,NaN,1697,Mats Karlsson,Kungsbacka MA,Sherco,Motion 60-,96
70,NaN,NaN,990,Johan Kjellson,FMCK Skövde,KTM,Motion 40-49,97
42,NaN,NaN,911,Robert Ahlin,Falköpings MK,KTM,Motion 40-49,98
22,NaN,NaN,513,Niklas Kedén,Hjo MK,KTM,Motion -39,99


## Save to new CSV.

In [25]:
folder_name = '../src/seeded'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
# Get the current date and time
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")

# Define the filename with the current timestamp
file_name = f"../src/seeded/{current_time}.csv"

# Save the DataFrame to a CSV file
try:
    seeded_df.to_csv(file_name, index=False, encoding='cp1252')  # Windows-1252 encoding
    print(f"Ranked data saved successfully to '{file_name}'")
except Exception as e:
    print(f"Error saving ranked data: {e}")

Ranked data saved successfully to '../src/seeded/2023-12-28_1059.csv'
